# Mapeamento dos municípios disponíveis na plataforma de venda de passagens Clickbus

In [9]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import geopandas as gpd

In [10]:
municipios = gpd.read_file("data/shapefile/centroide/municipios_centroide.shp")

In [11]:
# Criar o DataFrame 'cod_mun' com as colunas renomeadas
cod_mun = municipios[['NM_MUN', 'CD_MUN', 'SIGLA_UF']].rename(
    columns={
        'NM_MUN': 'nome_mun',
        'CD_MUN': 'cod_mun',
        'SIGLA_UF': 'sigla_uf'
    }
)

In [12]:
cod_mun.columns

Index(['nome_mun', 'cod_mun', 'sigla_uf'], dtype='object')

In [13]:
# Função para iniciar o driver do Edge
def iniciar_driver_edge():
    caminho_driver = r'webdriver/edgedriver_win64/msedgedriver.exe'
    service = EdgeService(executable_path=caminho_driver)
    options = Options()  # Adicione opções específicas se necessário
    driver = webdriver.Edge(service=service, options=options)
    return driver

In [14]:
def inserir_varias_origens(driver, cod_mun, limite=None):
    wait = WebDriverWait(driver, 10)

    # Lista para armazenar as origens e códigos selecionados com sucesso
    origens_selecionadas = []
    cod_origens_selecionados = []

    # Abrir a página da ClickBus
    driver.get("https://www.clickbus.com.br/")

    # Processar todos os itens de cod_mun ou limitar conforme o argumento fornecido
    total_municipios = len(cod_mun) if limite is None else min(limite, len(cod_mun))

    for i in range(total_municipios):
        # Aguardar o campo de origem estar presente e pronto para interação
        campo_origem = wait.until(EC.presence_of_element_located((By.ID, "origin")))

        # Limpar o campo completamente
        campo_origem.clear()
        campo_origem.send_keys(Keys.CONTROL + "a")  # Selecionar tudo
        campo_origem.send_keys(Keys.DELETE)  # Deletar o conteúdo

        # Criar a string concatenando 'nome_mun' e 'sigla_uf'
        valor_atual = f"{cod_mun['nome_mun'].iloc[i]}, {cod_mun['sigla_uf'].iloc[i]}"
        
        # Inserir o valor concatenado no campo de origem
        campo_origem.send_keys(valor_atual)

        # Esperar a lista de sugestões aparecer
        time.sleep(1)  # Verificação mais rápida, ajustável

        # Verificar as sugestões disponíveis
        try:
            lista_sugestoes = driver.find_element(By.ID, 'place-input-ul')
            sugestoes = lista_sugestoes.find_elements(By.CLASS_NAME, 'tt-suggestion')

            # Clicar na primeira sugestão válida
            for sugestao in sugestoes:
                if 'Nenhum resultado' not in sugestao.get_attribute('title'):
                    # Armazenar o texto exato da sugestão que será clicada
                    # Tentar clicar usando JavaScript para evitar o bloqueio de clique
                    driver.execute_script("arguments[0].click();", sugestao)
                    # Adicionar à lista de origens e códigos selecionados
                    origens_selecionadas.append(valor_atual)  # Salva o que foi pesquisado
                    cod_origens_selecionados.append(cod_mun['cod_mun'].iloc[i])
                    break
            else:
                # Caso nenhuma sugestão válida seja encontrada, não adicionar ao DataFrame
                print(f"Nenhuma sugestão válida encontrada para {valor_atual}.")
            
        except Exception as e:
            print(f"Erro ao selecionar a sugestão: {e}")
        
        # Mostrar progresso
        print(f"Pesquisado {i+1} de {total_municipios} municípios")

        # Pausa entre as iterações para garantir que cada valor seja processado adequadamente
        time.sleep(1)  # Ajustar o tempo de espera conforme necessário

    # Criar DataFrame com as origens e códigos selecionados, apenas para buscas bem-sucedidas
    od_rodo_origens = pd.DataFrame({
        'origem': origens_selecionadas,
        'cod_origem': cod_origens_selecionados
    })

    return od_rodo_origens

# Função principal
if __name__ == "__main__":

    driver = iniciar_driver_edge()
    
    try:
        # Definir o limite de municípios a serem processados, ou deixar None para todos
        limite_de_municipios = None  # Ajuste conforme necessário

        # Inserir todos os valores de cod_mun no campo de origem ou respeitar o limite
        od_rodo_origens = inserir_varias_origens(driver, cod_mun, limite=limite_de_municipios)

    finally:
        driver.quit()

Nenhuma sugestão válida encontrada para Alta Floresta D'Oeste, RO.
Pesquisado 1 de 5572 municípios
Nenhuma sugestão válida encontrada para Ariquemes, RO.
Pesquisado 2 de 5572 municípios
Nenhuma sugestão válida encontrada para Cabixi, RO.
Pesquisado 3 de 5572 municípios
Nenhuma sugestão válida encontrada para Cacoal, RO.
Pesquisado 4 de 5572 municípios
Nenhuma sugestão válida encontrada para Cerejeiras, RO.
Pesquisado 5 de 5572 municípios
Pesquisado 6 de 5572 municípios
Nenhuma sugestão válida encontrada para Corumbiara, RO.
Pesquisado 7 de 5572 municípios
Pesquisado 8 de 5572 municípios
Nenhuma sugestão válida encontrada para Espigão D'Oeste, RO.
Pesquisado 9 de 5572 municípios
Pesquisado 10 de 5572 municípios
Pesquisado 11 de 5572 municípios
Pesquisado 12 de 5572 municípios
Nenhuma sugestão válida encontrada para Machadinho D'Oeste, RO.
Pesquisado 13 de 5572 municípios
Nenhuma sugestão válida encontrada para Nova Brasilândia D'Oeste, RO.
Pesquisado 14 de 5572 municípios
Pesquisado 15 

In [15]:
od_rodo_origens.columns

Index(['origem', 'cod_origem'], dtype='object')

In [16]:
od_rodo_origens.to_csv('data/csv/od_rodo_origens.csv', index=False)